# Dependency code

In [ ]:
import imp
import os
import sys
import numpy as np
import glob
import cmdbench
import subprocess

fp, pathname, description = imp.find_module('benchmark', ['../lib'])
benchmark = imp.load_module('benchmark', fp, pathname, description)

os.environ['GASTROSNAPPER_CONFPATH'] = os.getcwd() + '/snapper_config'
os.environ['GASTROSNAPPER_REFPATH'] = os.getcwd() + '/reference_genomes'

# Summarize numpy array if it has more than 10 elements
np.set_printoptions(threshold=10)

# Set PICARD_JAR environment variable
picard_path = subprocess.run(['conda', 'run', '--name', 'snapperdb', 'which', 'picard'],
                             capture_output=True).stdout.strip().decode('utf-8')
picard_bin_dir = os.path.dirname(picard_path)
picard_link = os.readlink(picard_path)
picard_true_path = os.path.abspath(picard_bin_dir + '/' + picard_link)
picard_jar = os.path.dirname(picard_true_path) + '/picard.jar'

# snapperdb needs 'PICARD_JAR' environment variable set
print(picard_jar)
os.environ['PICARD_JAR'] = picard_jar

# Set GATK_JAR environment variable
gatk_path = subprocess.run(['conda', 'run', '--name', 'snapperdb', 'which', 'gatk3'],
                          capture_output=True).stdout.strip().decode('utf-8')
gatk_bin_dir = os.path.dirname(gatk_path)
gatk_link = os.readlink(gatk_path)
gatk_true_path = os.path.abspath(gatk_bin_dir + '/' + gatk_link)
gatk_jar = os.path.dirname(gatk_true_path) + '/GenomeAnalysisTK.jar'

# snapperdb needs 'GATK_JAR' environment variable set
print(gatk_jar)
os.environ['GATK_JAR'] = gatk_jar

# Software versions

In [ ]:
!conda run --name snapperdb samtools --version
!conda run --name snapperdb bcftools --version

In [ ]:
!conda run --name snapperdb bwa 2>&1 | grep Version

In [ ]:
!conda run --name snapperdb bowtie2 --version

In [ ]:
!conda run --name snapperdb gatk3 --version

In [ ]:
!conda run --name snapperdb picard CheckIlluminaDirectory --version 2>&1 | grep Version

# Benchmark

## Input data and constants

In [ ]:
input_dir = '../data/input-files/reads'
input_files_1 = [os.path.basename(f) for f in glob.glob(f'{input_dir}/*_1.fastq.gz')]
input_samples = [f.replace('_1.fastq.gz','') for f in input_files_1]
input_samples.sort()
print(input_samples)

sample_sizes = [1,10,20,30,40,50,60,70,80]
#sample_sizes = [3]

nprocs = 4
output_dir = 'output'
snapper_config = 'snapper_config'
reference_dir = 'reference_genomes'
config_file = f'{snapper_config}/config.txt'

reference_genome_name = '2011C-3609'

db_name = 'ecoli_snapperdb'
db_user = 'snapperdb'
db_pass = 'snapperdb'
db_host = 'localhost'

benchmark.create_folder_if_doesnt_exist(output_dir)
benchmark.create_folder_if_doesnt_exist(snapper_config)

## Setup database

### Postgres database

Create user with proper permissions

```
sudo su -l postgres
psql
create user snapperdb password 'snapperdb';
alter user snapperdb with superuser;
create database ecoli_snapperdb;
create database snapperdb; # Need snapperdb database to connect later
grant all on database ecoli_snapperdb to snapperdb;
```

Note, when logging into the database to test out, make sure to specify the hostname so it uses password authentication.

```
psql ecoli_snapperdb snapperdb --host localhost
```

### Create config file

In [ ]:
import textwrap

with open(config_file, 'w') as f:
    f.write(textwrap.dedent(f"""
    snpdb_name {db_name}
    reference_genome {reference_genome_name}
    pg_uname {db_user}
    pg_pword {db_pass}
    pg_host {db_host}
    depth_cutoff 10
    mq_cutoff 30
    ad_cutoff 0.9
    average_depth_cutoff 30
    mapper bwa
    mapper_threads 8
    variant_caller gatk
    variant_caller_threads 8
    """).strip() + '\n')

## Benchmark functions

In [ ]:
import psycopg2, glob, os
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

def reset_func():
    # Remove processed files for the reference genome
    for file in glob.glob(f"reference_genomes/{reference_genome_name}.*"):
        if(file.split(".")[-1] != "fa"):
            os.remove(file)
    
    # https://github.com/phe-bioinformatics/snapperdb#deleting-or-purging-your-database
    
    # Connect to the database, drop the whole database made by snapperdb and close the connection
    
    psql_conn = psycopg2.connect(f"user={db_user} host={db_host} password={db_pass}")
    psql_conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    
    psql_cursor = psql_conn.cursor()
    
    psql_cursor.execute(f"DROP DATABASE IF EXISTS {db_name};")
    
    psql_cursor.close()
    psql_conn.close()
    
def sampling_func(input_size):
    return input_samples[:input_size]

# Index

make_snpdb_command = {
    "use_parallel": False,
    "command": "conda run --name snapperdb run_snapperdb.py make_snpdb -c config.txt"
}

fastq_to_db_command = {
    "command": f'conda run --name snapperdb run_snapperdb.py fastq_to_db -c config.txt "{input_dir}/%_1.fastq.gz" "{input_dir}/%_2.fastq.gz"',
    "parallel_args": f"-j {nprocs} -I%"
}

update_distance_matrix_command = {
    "use_parallel": False,
    "command": "conda run --name snapperdb run_snapperdb.py update_distance_matrix -c config.txt"
}

update_clusters_command = {
    "use_parallel": False,
    "command": "conda run --name snapperdb run_snapperdb.py update_clusters -c config.txt"
}

# Query

get_strains_command = {
    "use_parallel": False,
    "command": "conda run --name snapperdb run_snapperdb.py get_strains -c config.txt"
}

# Benchmark

In [ ]:
multibench_results, debug_str = benchmark.multibench.multi_cmdbench({
        "index": [make_snpdb_command, fastq_to_db_command, update_distance_matrix_command, update_clusters_command],
        "query": [get_strains_command]
    }, reset_func = reset_func, iterations = 1, sampling_func = sampling_func, sample_sizes = sample_sizes, 
    benchmark_list_to_results=benchmark.benchmark_list_to_results, active_output_print = True
)

# Save and reload results

In [ ]:
save_path = "snapperdb-results.txt"

samples_per_sample_size = []
for sample_size in sample_sizes:
        samples_per_sample_size.append(input_samples[:sample_size])

benchmark.multibench.save_multibench_results(multibench_results, samples_per_sample_size, save_path)
multibench_results, samples_per_sample_size = benchmark.multibench.read_multibench_results(save_path)
print(samples_per_sample_size)

# Plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from pylab import rcParams
rcParams['figure.figsize'] = 15, 3

In [ ]:
# Indexing Plots
benchmark.multibench.plot_resources(multibench_results, sample_sizes, "index")

In [ ]:
# Querying Plots
benchmark.multibench.plot_resources(multibench_results, sample_sizes, "query")